# 2/14  1. Введение

ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

In [ ]:
import pandas as pd
import numpy as np

melb_dfc = pd.read_csv('data/melb_data_fe.csv')
melb_df = melb_dfc.copy()
melb_df.head()

In [ ]:
display(melb_df.info())

`csv`-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении Pandas автоматически определяет тип данных столбца. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.

Задание 1.1

Преобразуйте столбец `Date` в формат `datetime` и выделите квартал (`quarter`) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['Date'].dt.quarter.value_counts()


Задание 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных `category`, исключив из преобразования столбцы `Date`, `Rooms`, `Bedroom`, `Bathroom`, `Car`.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных `category`.

In [ ]:
unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(), melb_df[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(
    unique_list, columns=[
        'Colomns', 'Unique', 'Dtype']).sort_values(
            by='Unique', ignore_index=True)
        
display(unique_counts)


In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # Список столбцов, которые мы не берём во внимание
max_unique_count = 150 # Задаём максимальное число уникальных категорий
for col in melb_df.columns: # Цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # Проверяем условие
        melb_df[col] = melb_df[col].astype('category') # Преобразуем тип столбца
display(melb_df.info())

# 3/14  2. Сортировка данных в DataFrame

МЕТОД `SORT_VALUES()`

Основные параметры метода `sort_values()`

`by` — имя или список имён столбцов, по значениям которых производится сортировка.  
`axis` — ось, по которой производится сортировка (`0` — строки, `1` — столбцы). По умолчанию сортировка производится по строкам.  
`ascending` — сортировка по **возрастанию** (от меньшего к большему). По умолчанию параметр выставлен на `True`, для сортировки по **убыванию** (от большего к меньшему) необходимо выставить его на `False`.  
`ignore_index` — создаются ли новые индексы в таблице. По умолчанию выставлен на `False` и сохраняет индексы изначальной таблицы.  
`inplace` — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на `False`, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на `True`.

СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

In [ ]:
melb_df.sort_values(by='Price').head(10)

In [ ]:
melb_df.sort_values(by='Date', ascending=False).head(10)

СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр `by` в виде списка. При этом важно обращать внимание на порядок следования столбцов.

Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (`Distance`), а затем — по возрастанию цены объекта (`Price`). Для того чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов `Distance` и `Price` результирующей таблицы:

In [ ]:
melb_df.sort_values(
    by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым, то внутри такой группы производится сортировка по цене объекта.

Ради интереса попробуйте поменять порядок следования столбцов в параметре `by` метода `sort_values()` и сравните результат. 

In [ ]:
melb_df.sort_values(
    by=['Price', 'Distance']).loc[::10, ['Distance', 'Price']]

In [ ]:
ddd = {'counts': [np.random.randint(20) for i in range(20)], 'persent':[np.random.randint(20) for i in range(20)]}
ddd1 = pd.DataFrame(ddd)
ddd1.loc[2:19:5,'persent']# Со 2-го по 19-й, но каждый 5-й.

КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

Найдём информацию о таунхаусах (`Type`), проданных компанией (`SellerG`) `McGrath`, у которых коэффициент соотношения площадей здания и участка (`AreaRatio`) меньше `-0.8`. Результат отсортируем по дате продажи (`Date`) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр `ignore_index` на `True`. Для наглядности результата выберем из таблицы только столбцы `Data` и `AreaRatio`:

May option

In [ ]:
melb_df[(
    melb_df['Type'] == 'townhouse') & (
        melb_df['SellerG'] == 'McGrath') & (
            melb_df['AreaRatio'] <= -0.8)].sort_values(
                by=['Date', 'AreaRatio'], ascending=[
                    True, False], ignore_index=True).loc[
                        ::, ['Date', 'AreaRatio']]

Option SkillFactory

In [ ]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

ПРО ВАЖНОСТЬ КОПИРОВАНИЯ ПРИ СОРТИРОВКЕ С `inplace=Tru`e

Примечание. Старайтесь не сочетать фильтрацию и метод sort_values() с параметром inplace=True, так как в таком случае у вас возникнет предупреждение (warning) SettingWithCopyWarning: melb_df[melb_df['Rooms'] > 5].sort_values(inplace=True, by=['Rooms']):

C:\Users\Andrey\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingwithCopyWarning:

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ indexing. html#retu rning-a-view-versus-a-copy

    return func(*args, **kwargs)
    
Стоит отметить, что это не ошибка и код в таком случае отработает. Однако Pandas предупреждает вас о том, что при использовании такого кода дальнейшие результаты могут быть неожиданными.

Это предупреждение предназначено для обозначения операций «цепного присваивания». Это ситуация, когда вы пытаетесь напрямую изменить подмножество исходных данных. В нашем случае мы пытаемся отсортировать данные с заменой (об этом говорит параметр inplace=True) исходной таблицы на отсортированную.

Чтобы не возникало подобных конфликтов, необходимо использовать метод copy() для явного создания копии отфильтрованного подмножества исходных данных и работать уже с ней (производить сортировку):

filtered = melb_df[melb_df['Rooms'] > 5].copy()

filtered.sort_values(inplace=True, by=['Rooms'])

filtered.head()

Задание 2.2

Произведите сортировку столбца `AreaRatio` по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке `1558`? Ответ округлите до целого числа.

In [ ]:
melb_df.sort_values(
    by='AreaRatio', ascending=False, ignore_index=True).loc[
        1558, 'BuildingArea']

Задание 2.3

Найдите таунхаусы (`Type`) с количеством жилых комнат (`Rooms`) больше `2`. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (`MeanRoomsSquare`). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [ ]:
melb_df[(melb_df['Type'] == 'townhouse') & (melb_df[
    'Rooms'] > 2)].sort_values(by=[
        'Rooms', 'MeanRoomsSquare'], ascending=[
            True, False], ignore_index=True).loc[18, 'Price']

# 4/14  3. Группировка данных в DataFrame

МЕТОД `GROUPBY()`

В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод `groupby()`.

Основные параметры метода `groupby()`

`by` — имя или список имён столбцов, по которым производится группировка.
`axis` — ось, по которой производится группировка (`0` — строки, `1` — столбцы). По умолчанию группировка производится по строкам.
`as_index` — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на `True`.

Метод `groupby()` возвращает объект `DataFrameGroupBy`, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 

Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (`mean`, `median`, `sum` и т. д.), чтобы рассчитывать показатели внутри каждой группы.

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

In [ ]:
melb_df.groupby(by='Type').mean(numeric_only=True)

Мы получили таблицу, на пересечении строк и столбцов которой находятся средние значения каждого числового признака в наших данных.

**Примечание**. Если мы хотим видеть тип объекта в качестве отдельного столбца таблицы, мы можем выставить параметр `as_index` на `False`:

In [ ]:
melb_df.groupby(by='Type', as_index=False).mean(numeric_only=True)

Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:

In [ ]:
melb_df.groupby('Type')['Price'].mean()

**Примечание**. Обратите внимание, что, так как мы считаем только один показатель (среднее) для одного столбца, в результате мы получаем объект `Series`.

Теперь давайте выясним, какие регионы (`Regionname`) наиболее удалены от центра Мельбурна.

In [ ]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом `agg()`, который принимает список строк с названиями агрегаций.

Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (`Price`), сгруппированных по номеру месяца продажи (`MonthSale`). Результат отсортируем по количеству продаж в порядке убывания:

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(['count', 'max', 'mean']).sort_values(by='count', ascending=False)

**Примечание**. Обратите внимание, что, так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект `DataFrame`.

**Примечание**. Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом `agg()`, передав в качестве его параметра строку '`describe`':

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')# = ... .describe()

После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода `agg()` строку '`nunique`'. 

Более того, метод `agg()` поддерживает использование и других функций. Передадим дополнительно встроенную функцию `set`, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

In [ ]:
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set]).sort_values(by='nunique', ascending=False)

Задание 3.1

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [ ]:
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

Задание 3.2

Какой регион имеет наименьшее стандартное отклонение по географической широте (`Lattitude`)?
В качестве ответа запишите название этого региона.

In [ ]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

Задание 3.3

Какая риелторская компания (`SellerG`) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
Для ответа на этот вопрос рассчитайте сумму продаж (`Price`) каждой компании в заданный период.
Не забудьте перевести даты в формат `datetime`.

In [ ]:
melb_df['Date']

In [ ]:
data_from = (melb_df['Date'] >= pd.to_datetime('2017.05.01'))
data_to = (melb_df['Date'] <= pd.to_datetime('2017.09.01'))
melb_df[data_from & data_to].groupby('SellerG')['Price'].sum().sort_values().index[0]

# 5/14  4. Сводные таблицы

МЕТОД `GROUPBY` КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

Mы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:

In [ ]:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (`Type`). Для этого в параметрах метода `groupby()` укажем список из нескольких интересующих нас столбцов.

In [ ]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов: первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется иерархической. Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.

Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде `Series` с иерархическими индексами, к результату чаще всего применяют метод `unstack()`, который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [ ]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

МЕТОД `PIVOT_TABLE` ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — `pivot_table()`.

Основные параметры метода `pivot_table()`

`values` — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;  
`index` — имя столбца, значения которого станут строками сводной таблицы;  
`columns` — имя столбца, значения которого станут столбцами сводной таблицы;  
`aggfunc` — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, '`mean`');  
`fill_value` — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

Давайте построим ту же самую таблицу, но уже с использованием метода `pivot_table`. В качестве параметра `values` укажем столбец `Price`, в качестве индексов сводной таблицы возьмём `Rooms`, а в качестве столбцов — `Type`. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение `0`. Финальный результат для наглядности вывода округлим с помощью метода `round()` до целых.

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',    fill_value=0
).round()

А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (`Regionname`), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (`Weekend`), который равен `1`, если день был выходным, и `0` — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (`Landsize`) от типа объекта (`Type`) и его региона (`Regionname`). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент `aggfunc` список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на `0`:

In [ ]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

Обратите внимание на добавление дополнительных индексов столбцов `median` и `mean`. Здесь медианное и среднее значения рассчитаны отдельно для каждой комбинации признаков.

МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр `index` или параметр `columns`.

Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (`Method`) и типа объекта (`Type`), по столбцам — наименование региона (`Regionname`), а на пересечении строк и столбцов будет стоять медианная цена объекта (`Price`):

In [ ]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
    )

ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.

Запишем сводную таблицу, которую мы создавали ранее в переменную `pivot`:

In [ ]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns

В результате мы получаем объект `MultiIndex`. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

Так, из таблицы `pivot` мы можем получить средние значения площадей участков для типа здания `unit`, просто последовательно обратившись по имени столбцов:

In [ ]:
pivot['mean']['unit']

Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа `house` меньше их медианной площади, то мы можем найти их следующим образом:

In [ ]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом `index` и обернуть результат в список:

In [ ]:
print(list(filtered_pivot.index))
# ['Southern Metropolitan', 'Western Metropolitan']

Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:

In [ ]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

Если посмотреть на индексы `Series`, можно увидеть, что они являются мультииндексами:

In [ ]:
print(mser.index)

Аналогично создаются `DataFrame` со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр `columns` при инициализации таблицы:

In [ ]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

Задание 4.2

Составьте сводную таблицу, которая показывает зависимость медианной площади (`BuildingArea`) здания от типа объекта недвижимости (`Type`) и количества жилых комнат в доме (`Rooms`). Для какой комбинации признаков площадь здания наибольшая?
В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

In [ ]:
melb_df.pivot_table(
        values='BuildingArea', 
        index='Type', 
        columns='Rooms', 
        aggfunc='median'
)

Задание 4.3

Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (`Price`) от риелторского агентства (`SellerG`) и типа здания (`Type`).
Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа `unit` максимальна. В качестве ответа запишите название этого агентства.

In [ ]:
median_price = melb_df.pivot_table(
        values='Price', 
        index='SellerG', 
        columns='Type', 
        aggfunc='median'
)
median_price.sort_values(by='unit', ascending=False)

# 6/14  5. Объединение DataFrame: знакомимся с новыми данными

С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?

В этой части модуля мы будем работать с популярным датасетом `MovieLens`, в котором собраны логи некоторой рекомендательной системы фильмов.

Наши данные представляют собой четыре таблицы:

`ratings1` и `ratings2` — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.  
- `userId` — уникальный идентификатор пользователя, который выставил оценку;   
- `movieId` — уникальный идентификатор фильма;  
- `rating` — рейтинг фильма.  

`dates` — таблица с датами выставления всех оценок.  

`movies` — таблица с информацией о фильмах.  
- `movieId` — уникальный идентификатор фильма;
- `title` — название фильма и год его выхода;
- `genres` — жанры фильма.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1
Склеим таблицы `ratings1` и `ratings2` в единую структуру.

2
К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.

3
Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?

Задание 5.1

Значения из какого столбца таблиц `ratings1` и `ratings2` можно расшифровать с помощью таблицы `movies`?

In [ ]:
dates_df = pd.read_csv('data/movies_data/dates.csv').copy()
movies_df = pd.read_csv('data/movies_data/movies.csv').copy()
ratings1_df = pd.read_csv('data/movies_data/ratings1.csv').copy()
ratings2_df = pd.read_csv('data/movies_data/ratings2.csv').copy()
#dates_df
movies_df
#ratings1_df
#ratings2_df

Задание 5.2

Сколько уникальных фильмов представлено в таблице `movies`?

In [ ]:
movies_df['title'].nunique

Задание 5.3

Сколько уникальных пользователей в таблице `ratings1`?

In [ ]:
ratings1_df.nunique()

Задание 5.4

В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу `dates`.

In [ ]:
dates_df['date'] = pd.to_datetime(dates_df['date'])
dates_df['date'].dt.year.value_counts()